# PFT_GEM Tutorial: Geometric Expansion Model for Tumor Displacement

This notebook demonstrates how to use PFT_GEM (Posterior Fossa Tumor - Geometric Expansion Model) to simulate tumor-induced brain displacement using a computationally efficient geometric expansion approach.

## Overview

PFT_GEM provides a simplified alternative to finite element methods for modeling how brain tumors displace surrounding tissue. The model is particularly useful for:

- Rapid estimation of displacement fields
- Visualization of tumor mass effect
- Integration with diffusion MRI for biophysical constraints
- Research on posterior fossa tumors (cerebellum, brainstem)

## Contents

1. [Setup and Imports](#1.-Setup-and-Imports)
2. [Creating a Tumor Model](#2.-Creating-a-Tumor-Model)
3. [Computing Displacement Fields](#3.-Computing-Displacement-Fields)
4. [Adding Biophysical Constraints from DTI](#4.-Adding-Biophysical-Constraints-from-DTI)
5. [Visualizing Results](#5.-Visualizing-Results)
6. [Working with SUIT Templates](#6.-Working-with-SUIT-Templates)
7. [Saving and Loading Results](#7.-Saving-and-Loading-Results)
8. [Advanced: Analyzing Strain and Volume Changes](#8.-Advanced:-Analyzing-Strain-and-Volume-Changes)

## 1. Setup and Imports

In [ ]:
# Install PFT_GEM if not already installed
# !pip install -e ..

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import PFT_GEM modules
from pft_gem import GeometricExpansionModel, DisplacementField, BiophysicalConstraints
from pft_gem.core.geometric_model import TumorParameters, ModelParameters
from pft_gem.core.constraints import DTIData, create_synthetic_dti
from pft_gem.io import SUITTemplateLoader
from pft_gem.visualization import (
    plot_displacement_field,
    plot_displacement_magnitude,
    plot_vector_field,
    plot_jacobian,
    plot_tumor_displacement_profile,
    VisualizationConfig
)
from pft_gem.utils import create_spherical_tumor_mask, create_ellipsoidal_tumor_mask

print("PFT_GEM modules loaded successfully!")

## 2. Creating a Tumor Model

First, we define the tumor geometry and model parameters. The tumor is characterized by:
- **Center**: Location in mm or voxel coordinates
- **Radius**: Tumor size in mm
- **Shape**: Spherical or ellipsoidal

In [ ]:
# Define computational grid
grid_shape = (128, 128, 64)  # Volume dimensions in voxels
voxel_size = (1.0, 1.0, 1.0)  # Voxel size in mm

# Define tumor parameters
# A posterior fossa tumor centered in the cerebellum
tumor_params = TumorParameters(
    center=(64.0, 64.0, 32.0),  # Center coordinates in mm
    radius=15.0,                 # 15mm radius (3cm diameter tumor)
    shape="spherical"           # or "ellipsoidal"
)

# Define model parameters
model_params = ModelParameters(
    decay_exponent=2.0,        # How quickly displacement decays with distance
    boundary_stiffness=0.8,    # Stiffness factor at anatomical boundaries
    csf_damping=0.1,           # Damping in CSF regions
    max_displacement=15.0,     # Maximum allowed displacement (mm)
    tissue_modulation=True,    # Apply tissue-specific modulation
    use_dti_constraints=True,  # Use DTI-derived constraints
    smoothing_sigma=1.0        # Gaussian smoothing (mm)
)

print(f"Tumor center: {tumor_params.center}")
print(f"Tumor radius: {tumor_params.radius} mm")
print(f"Grid shape: {grid_shape}")
print(f"Physical extent: {tuple(g*v for g,v in zip(grid_shape, voxel_size))} mm")

In [ ]:
# Create the geometric expansion model
model = GeometricExpansionModel(
    tumor_params=tumor_params,
    model_params=model_params,
    grid_shape=grid_shape,
    voxel_size=voxel_size
)

print("Model created!")
print(f"Configuration: {model.to_dict()}")

## 3. Computing Displacement Fields

The geometric expansion model computes displacement based on:

$$u(r) = u_0 \cdot \left(\frac{R}{r}\right)^\alpha \cdot f(\text{tissue}) \cdot g(\text{boundaries})$$

Where:
- $u_0$: displacement at tumor boundary
- $R$: tumor radius
- $r$: distance from tumor center
- $\alpha$: decay exponent

In [ ]:
# Compute displacement field
# The displacement represents how much each point would move due to tumor expansion
displacement = model.compute_displacement_field(tumor_expansion=5.0)  # 5mm expansion

print(f"Displacement field shape: {displacement.shape}")
print(f"Components: (x, y, z) displacement at each voxel")

# Compute magnitude
magnitude = np.linalg.norm(displacement, axis=-1)
print(f"\nDisplacement statistics:")
print(f"  Max displacement: {magnitude.max():.2f} mm")
print(f"  Mean displacement: {magnitude.mean():.2f} mm")
print(f"  Non-zero voxels: {np.sum(magnitude > 0.01)}/{magnitude.size}")

In [ ]:
# Visualize the displacement magnitude in three views
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Create tumor mask for overlay
tumor_mask = create_spherical_tumor_mask(
    grid_shape, 
    tumor_params.center, 
    tumor_params.radius
)

# Sagittal view (X slice)
slice_x = int(tumor_params.center[0])
im0 = axes[0].imshow(magnitude[slice_x, :, :].T, origin='lower', cmap='hot')
axes[0].contour(tumor_mask[slice_x, :, :].T, levels=[0.5], colors='cyan', linewidths=2)
axes[0].set_title(f'Sagittal (x={slice_x})')
axes[0].set_xlabel('Y (voxels)')
axes[0].set_ylabel('Z (voxels)')

# Coronal view (Y slice)
slice_y = int(tumor_params.center[1])
im1 = axes[1].imshow(magnitude[:, slice_y, :].T, origin='lower', cmap='hot')
axes[1].contour(tumor_mask[:, slice_y, :].T, levels=[0.5], colors='cyan', linewidths=2)
axes[1].set_title(f'Coronal (y={slice_y})')
axes[1].set_xlabel('X (voxels)')
axes[1].set_ylabel('Z (voxels)')

# Axial view (Z slice)
slice_z = int(tumor_params.center[2])
im2 = axes[2].imshow(magnitude[:, :, slice_z].T, origin='lower', cmap='hot')
axes[2].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[2].set_title(f'Axial (z={slice_z})')
axes[2].set_xlabel('X (voxels)')
axes[2].set_ylabel('Y (voxels)')

fig.colorbar(im2, ax=axes, label='Displacement (mm)', shrink=0.8)
fig.suptitle('Displacement Magnitude (tumor boundary shown in cyan)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Adding Biophysical Constraints from DTI

Diffusion MRI (DTI) provides information about tissue microstructure that can be used to modulate the displacement field:

- **Fractional Anisotropy (FA)**: Higher FA indicates more organized tissue (typically stiffer)
- **Mean Diffusivity (MD)**: Lower MD indicates denser tissue (typically stiffer)

In [ ]:
# Create synthetic DTI data for demonstration
# In practice, you would load real DTI maps from NIfTI files
dti_data = create_synthetic_dti(
    shape=grid_shape,
    tumor_center=(int(tumor_params.center[0]), 
                  int(tumor_params.center[1]), 
                  int(tumor_params.center[2])),
    tumor_radius=tumor_params.radius
)

print(f"DTI data created:")
print(f"  FA range: [{dti_data.fa.min():.3f}, {dti_data.fa.max():.3f}]")
print(f"  MD range: [{dti_data.md.min()*1000:.2f}, {dti_data.md.max()*1000:.2f}] x 10^-3 mm²/s")

In [ ]:
# Visualize DTI maps
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# FA map
im0 = axes[0].imshow(dti_data.fa[:, :, slice_z].T, origin='lower', cmap='viridis', vmin=0, vmax=1)
axes[0].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[0].set_title('Fractional Anisotropy (FA)')
plt.colorbar(im0, ax=axes[0])

# MD map
im1 = axes[1].imshow(dti_data.md[:, :, slice_z].T * 1000, origin='lower', cmap='plasma')
axes[1].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[1].set_title('Mean Diffusivity (MD × 10³)')
plt.colorbar(im1, ax=axes[1], label='mm²/s × 10³')

# Principal eigenvector direction (color-coded)
# RGB coding: R=x, G=y, B=z
v1_rgb = np.abs(dti_data.v1[:, :, slice_z, :])
v1_rgb = v1_rgb / (v1_rgb.max() + 1e-10)
axes[2].imshow(np.transpose(v1_rgb, (1, 0, 2)), origin='lower')
axes[2].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='white', linewidths=2)
axes[2].set_title('Principal Direction (RGB=XYZ)')

fig.suptitle('Synthetic DTI Maps (tumor boundary shown)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Create biophysical constraints from DTI
constraints = BiophysicalConstraints(dti_data)

# Compute stiffness map
stiffness = constraints.compute_stiffness_map(normalize=True)
print(f"Stiffness map range: [{stiffness.min():.3f}, {stiffness.max():.3f}]")

# Compute compliance (inverse of stiffness)
compliance = constraints.compute_compliance_map()

# Visualize stiffness
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im0 = axes[0].imshow(stiffness[:, :, slice_z].T, origin='lower', cmap='coolwarm')
axes[0].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[0].set_title('Tissue Stiffness (from DTI)')
plt.colorbar(im0, ax=axes[0], label='Relative stiffness')

im1 = axes[1].imshow(compliance[:, :, slice_z].T, origin='lower', cmap='coolwarm_r')
axes[1].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[1].set_title('Tissue Compliance (1/stiffness)')
plt.colorbar(im1, ax=axes[1], label='Relative compliance')

plt.tight_layout()
plt.show()

In [ ]:
# Apply DTI constraints to the model and recompute displacement
model.set_dti_constraints(dti_data.fa, dti_data.md)

# Recompute displacement with DTI modulation
displacement_dti = model.compute_displacement_field(tumor_expansion=5.0)
magnitude_dti = np.linalg.norm(displacement_dti, axis=-1)

print(f"With DTI constraints:")
print(f"  Max displacement: {magnitude_dti.max():.2f} mm")
print(f"  Mean displacement: {magnitude_dti.mean():.2f} mm")

In [ ]:
# Compare displacement with and without DTI constraints
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Without DTI
im0 = axes[0].imshow(magnitude[:, :, slice_z].T, origin='lower', cmap='hot', vmax=magnitude.max())
axes[0].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[0].set_title('Without DTI Constraints')
plt.colorbar(im0, ax=axes[0], label='mm')

# With DTI
im1 = axes[1].imshow(magnitude_dti[:, :, slice_z].T, origin='lower', cmap='hot', vmax=magnitude.max())
axes[1].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[1].set_title('With DTI Constraints')
plt.colorbar(im1, ax=axes[1], label='mm')

# Difference
diff = magnitude_dti - magnitude
vmax = np.abs(diff).max()
im2 = axes[2].imshow(diff[:, :, slice_z].T, origin='lower', cmap='RdBu_r', vmin=-vmax, vmax=vmax)
axes[2].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[2].set_title('Difference (DTI - baseline)')
plt.colorbar(im2, ax=axes[2], label='mm')

fig.suptitle('Effect of DTI Constraints on Displacement', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Visualizing Results

PFT_GEM includes several visualization tools for analyzing displacement fields.

In [ ]:
# Plot displacement vectors
fig = plot_vector_field(
    displacement_dti,
    slice_idx=slice_z,
    axis=2,
    config=VisualizationConfig(vector_density=6, vector_scale=2.0)
)
plt.title('Displacement Vector Field (axial slice)')
plt.show()

In [ ]:
# Plot displacement profile as a function of distance from tumor
fig = plot_tumor_displacement_profile(
    displacement_dti,
    tumor_center=(int(tumor_params.center[0]), 
                  int(tumor_params.center[1]), 
                  int(tumor_params.center[2])),
    directions=["x", "y", "z", "radial"],
    max_distance=50
)
plt.axvline(x=tumor_params.radius, color='gray', linestyle='--', label='Tumor boundary')
plt.legend()
plt.show()

## 6. Working with SUIT Templates

The SUIT (Spatially Unbiased Infratentorial Template) provides standardized reference data for cerebellum and brainstem analysis.

In [ ]:
# Create a synthetic template for demonstration
# In practice, you would load real SUIT templates
template_data = SUITTemplateLoader.create_synthetic_template(
    shape=grid_shape,
    voxel_size=voxel_size
)

print(f"Template shape: {template_data.template.shape}")
print(f"Mask shape: {template_data.mask.shape}")
print(f"Atlas labels: {template_data.labels}")

In [ ]:
# Visualize the synthetic template
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Template
axes[0].imshow(template_data.template[:, :, slice_z].T, origin='lower', cmap='gray')
axes[0].set_title('Synthetic T1 Template')

# Mask
axes[1].imshow(template_data.mask[:, :, slice_z].T, origin='lower', cmap='Blues')
axes[1].set_title('Brain Mask')

# Atlas
axes[2].imshow(template_data.atlas[:, :, slice_z].T, origin='lower', cmap='tab10')
axes[2].set_title('Parcellation Atlas')

plt.tight_layout()
plt.show()

In [ ]:
# Apply displacement to warp the template
# Wrap displacement in DisplacementField class for advanced operations
from pft_gem.core.displacement import DisplacementField, FieldMetadata

field = DisplacementField(
    displacement_dti,
    metadata=FieldMetadata(shape=grid_shape, voxel_size=voxel_size)
)

# Warp the template
warped_template = field.apply_to_image(template_data.template, order=1)

print(f"Warped template shape: {warped_template.shape}")

In [ ]:
# Compare original and warped template
from pft_gem.visualization import plot_slice_comparison

fig = plot_slice_comparison(
    template_data.template,
    warped_template,
    slice_idx=slice_z,
    axis=2,
    difference=True
)
fig.suptitle('Template Deformation by Tumor', fontsize=14)
plt.show()

## 7. Saving and Loading Results

In [ ]:
# Example: Saving displacement field to NIfTI (uncomment to run)
# from pft_gem.io import save_displacement_field
#
# save_displacement_field(
#     displacement_dti,
#     'displacement_field.nii.gz',
#     voxel_size=voxel_size
# )
# print("Displacement field saved!")

# Example: Loading displacement field
# from pft_gem.io import load_displacement_field
# loaded_disp = load_displacement_field('displacement_field.nii.gz')

print("Note: Uncomment the code above to save/load displacement fields")

In [ ]:
# Export model configuration
import json

config = model.to_dict()
print("Model configuration:")
print(json.dumps(config, indent=2))

# Save to file (uncomment to run)
# with open('model_config.json', 'w') as f:
#     json.dump(config, f, indent=2)

## 8. Advanced: Analyzing Strain and Volume Changes

The displacement field can be analyzed to understand local tissue deformation.

In [ ]:
# Compute Jacobian determinant (local volume change)
jacobian = model.compute_jacobian_determinant()

print(f"Jacobian statistics:")
print(f"  Min: {jacobian.min():.3f} (max compression)")
print(f"  Max: {jacobian.max():.3f} (max expansion)")
print(f"  Mean: {jacobian.mean():.3f}")
print(f"  Voxels with J < 0 (folding): {np.sum(jacobian < 0)}")

In [ ]:
# Visualize Jacobian
fig = plot_jacobian(
    jacobian,
    slice_idx=slice_z,
    axis=2,
    symmetric_scale=True
)
plt.show()

In [ ]:
# Compute strain tensor
strain = model.compute_strain_field()
print(f"Strain tensor shape: {strain.shape}")

# Compute principal strains (eigenvalues)
principal_strains = np.linalg.eigvalsh(strain)
max_strain = np.max(np.abs(principal_strains), axis=-1)

print(f"Maximum principal strain: {max_strain.max():.4f}")

In [ ]:
# Get comprehensive field statistics
stats = field.statistics(mask=template_data.mask)
print("Displacement field statistics (within brain mask):")
for category, values in stats.items():
    print(f"\n{category.upper()}:")
    for key, val in values.items():
        print(f"  {key}: {val:.4f}")

## Summary

In this tutorial, we covered:

1. **Creating a geometric expansion model** with tumor parameters
2. **Computing displacement fields** showing tissue deformation
3. **Adding DTI-derived biophysical constraints** for more realistic results
4. **Visualizing** displacement magnitude, vectors, and profiles
5. **Working with SUIT templates** for anatomical reference
6. **Saving and loading** displacement fields in NIfTI format
7. **Analyzing strain and volume changes** from the deformation

### Key Advantages of PFT_GEM

- **Computational efficiency**: Much faster than FEM approaches
- **Biophysical plausibility**: DTI constraints ensure realistic tissue behavior
- **Easy integration**: NIfTI format compatible with standard neuroimaging tools
- **Flexibility**: Supports spherical and ellipsoidal tumors, various parameter tuning

### Next Steps

- Load real SUIT templates for anatomically accurate simulations
- Use patient-specific DTI data for personalized modeling
- Explore parameter sensitivity for your specific application
- Compare results with FEM-based approaches (PFT_FEM)